# Capstone: Portfolio Optimization

In [1]:
#Base
import numpy as np
import pandas as pd
from datetime import datetime

#Visualization
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

#Data Optimization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

#Analysis
import statsmodels.api as sm
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.optimize import minimize
import cvxpy as cp


# 1. Data Wrangling

### 1.1 Upload the csv files

In [2]:
df=pd.read_csv('database2.csv')
portfolio_forecast=pd.read_csv('selected_stocks(forecast)2.csv')
portfolio_historical=pd.read_csv('selected_stocks(historical)2.csv')

In [3]:
portfolio_forecast.set_index('Stock',inplace=True)
portfolio_historical.set_index('Stock',inplace=True)

In [4]:
df['date'] = pd.to_datetime(df['date'])
df = df[(df['date']>'2019-02-07')&(df['date']<='2020-02-07')]
stock_name = [col for col in df.columns if col.endswith('_close')]
volume_columns = [col for col in df.columns if col.endswith('_volume')]
df = df.drop(columns=volume_columns)
general_columns = ['sp5avg','timing','inf_close','spr_close','int_close','dj_close','vix_close']
df = df.drop(columns=general_columns)
stock_df = df.copy()
stock_df = stock_df.set_index('date')

In [5]:
maxstock_forecast=portfolio_forecast[:1]
maxstock_historical=portfolio_historical[:1]

In [6]:
maxstock_historical

,Normalized_Weight
Stock,
NEE_close,0.196263


In [7]:
maxstock_forecast

,Normalized_Weight
Stock,
AMT_close,0.116088


In [8]:
# Create a DataFrame with only 'sp5t_close' and a Normalized_Weight of 1
maxstock_forecast = {
    'Stock': ['AMT_close'],
    'Normalized_Weight': [1]
}
maxstock_forecast = pd.DataFrame(maxstock_forecast).set_index('Stock')

In [9]:
# Create a DataFrame with only 'sp5t_close' and a Normalized_Weight of 1
maxstock_historical = {
    'Stock': ['NEE_close'],
    'Normalized_Weight': [1]
}
maxstock_historical = pd.DataFrame(maxstock_historical).set_index('Stock')

In [10]:
# Create a DataFrame with only 'sp5t_close' and a Normalized_Weight of 1
sp5t = {
    'Stock': ['sp5t_close'],
    'Normalized_Weight': [1]
}
sp5 = pd.DataFrame(sp5t).set_index('Stock')

In [11]:
# Calculate the daily returns for each stock
returns_df = stock_df.pct_change().dropna()

In [12]:
# Filter the 'returns_df' to contain only the stocks present in the 'weights_df'
portfolio_forecast_df = returns_df.loc[:, portfolio_forecast.index]
# Calculate the weighted daily returns
portfolio_forecast_returns_df = portfolio_forecast_df.mul(portfolio_forecast['Normalized_Weight'])
portfolio_forecast_daily_performance = portfolio_forecast_returns_df.sum(axis=1)
# Calculate the cumulative performance as if $1000 was invested on day 1
portfolio_forecast_cumulative = (1 + portfolio_forecast_daily_performance).cumprod() * 1000


In [36]:
# Filter the 'returns_df' to contain only the stocks present in the 'weights_df'
portfolio_historical_df = returns_df.loc[:, portfolio_historical.index]
# Calculate the weighted daily returns
portfolio_historical_returns_df = portfolio_historical_df.mul(portfolio_historical['Normalized_Weight'])
portfolio_historical_daily_performance = portfolio_historical_returns_df.sum(axis=1)
# Calculate the cumulative performance as if $1000 was invested on day 1
portfolio_historical_cumulative = (1 + portfolio_historical_daily_performance).cumprod() * 1000

In [14]:
# Filter the 'returns_df' to contain only the stocks present in the 'weights_df'
maxstock_forecast_df = returns_df.loc[:, maxstock_forecast.index]
# Calculate the weighted daily returns
maxstock_forecast_returns_df = maxstock_forecast_df.mul(maxstock_forecast['Normalized_Weight'])
maxstock_forecast_daily_performance = maxstock_forecast_returns_df.sum(axis=1)
# Calculate the cumulative performance as if $1000 was invested on day 1
maxstock_forecast_cumulative = (1 + maxstock_forecast_daily_performance).cumprod() * 1000

In [15]:
# Filter the 'returns_df' to contain only the stocks present in the 'weights_df'
maxstock_historical_df = returns_df.loc[:, maxstock_historical.index]
# Calculate the weighted daily returns
maxstock_historical_returns_df = maxstock_historical_df.mul(maxstock_historical['Normalized_Weight'])
maxstock_historical_daily_performance = maxstock_historical_returns_df.sum(axis=1)
# Calculate the cumulative performance as if $1000 was invested on day 1
maxstock_historical_cumulative = (1 + maxstock_historical_daily_performance).cumprod() * 1000

In [16]:
# Filter the 'returns_df' to contain only the stocks present in the 'weights_df'
sp5_df = returns_df.loc[:, sp5.index]
# Calculate the weighted daily returns
sp5_returns_df = sp5_df.mul(sp5['Normalized_Weight'])
sp5_daily_performance = sp5_returns_df.sum(axis=1)
# Calculate the cumulative performance as if $1000 was invested on day 1
sp5_cumulative = (1 + sp5_daily_performance).cumprod() * 1000

In [18]:
# Define colors for each trace with adjusted opacity
color_portfolio_forecast = '#F58518'    
color_portfolio_historical = '#4C78A8'  
color_maxstock_forecast = '#F58518'     
color_maxstock_historical = '#4C78A8'   
color_sp500 = '#BAB0AC'                

# Plot the portfolio daily performance
fig = go.Figure()
fig.add_trace(go.Scatter(x=portfolio_forecast_cumulative.index, y=portfolio_forecast_cumulative.values,
                         mode='lines', name='portfolio_forecast', line=dict(color=color_portfolio_forecast)))
fig.add_trace(go.Scatter(x=portfolio_historical_cumulative.index, y=portfolio_historical_cumulative.values,
                         mode='lines', name='portfolio_historical', line=dict(color=color_portfolio_historical)))
fig.add_trace(go.Scatter(x=maxstock_forecast_cumulative.index, y=maxstock_forecast_cumulative.values,
                         mode='lines', name='maxstock_forecast', line=dict(color=color_maxstock_forecast, dash='dot')))
fig.add_trace(go.Scatter(x=maxstock_historical_cumulative.index, y=maxstock_historical_cumulative.values,
                         mode='lines', name='maxstock_historical', line=dict(color=color_maxstock_historical, dash='dot')))
fig.add_trace(go.Scatter(x=sp5_cumulative.index, y=sp5_cumulative.values,
                         mode='lines', name='s&p500', line=dict(color=color_sp500)))
fig.update_layout(title='Portfolio Daily Performance (2019)',
                  xaxis_title='Date',
                  yaxis_title='Daily Performance',
                  xaxis=dict(showline=True, showgrid=False),
                  yaxis=dict(showline=True, showgrid=True))

fig.show()

In [40]:
# Step 1: Calculate the Portfolio's Average Daily Return
average_daily_return = portfolio_forecast_daily_performance.mean()

# Step 2: Calculate the Portfolio's Volatility (Standard Deviation)
portfolio_volatility = portfolio_forecast_daily_performance.std()

# Step 3: Calculate the Total Return of the Portfolio
total_return = portfolio_forecast_cumulative[-1] / 1000 - 1

# Print the results
print("Portfolio Total Return: {:.4f}".format(total_return))
print("Portfolio Average Daily Return: {:.4f}".format(average_daily_return))
print("Portfolio Volatility (Standard Deviation): {:.4f}".format(portfolio_volatility))

Portfolio Total Return: 0.2508
Portfolio Average Daily Return: 0.0006
Portfolio Volatility (Standard Deviation): 0.0050


In [41]:
# Step 1: Calculate the Portfolio's Average Daily Return
average_daily_return = portfolio_historical_daily_performance.mean()

# Step 2: Calculate the Portfolio's Volatility (Standard Deviation)
portfolio_volatility = portfolio_historical_daily_performance.std()

# Step 3: Calculate the Total Return of the Portfolio
total_return = portfolio_historical_cumulative[-1] / 1000 - 1

# Print the results
print("Portfolio Total Return: {:.4f}".format(total_return))
print("Portfolio Average Daily Return: {:.4f}".format(average_daily_return))
print("Portfolio Volatility (Standard Deviation): {:.4f}".format(portfolio_volatility))

Portfolio Total Return: 0.1982
Portfolio Average Daily Return: 0.0005
Portfolio Volatility (Standard Deviation): 0.0053


In [42]:
# Step 1: Calculate the Portfolio's Average Daily Return
average_daily_return = maxstock_forecast_daily_performance.mean()

# Step 2: Calculate the Portfolio's Volatility (Standard Deviation)
portfolio_volatility = maxstock_forecast_daily_performance.std()

# Step 3: Calculate the Total Return of the Portfolio
total_return = maxstock_forecast_cumulative[-1] / 1000 - 1

# Print the results
print("Portfolio Total Return: {:.4f}".format(total_return))
print("Portfolio Average Daily Return: {:.4f}".format(average_daily_return))
print("Portfolio Volatility (Standard Deviation): {:.4f}".format(portfolio_volatility))

Portfolio Total Return: 0.3846
Portfolio Average Daily Return: 0.0009
Portfolio Volatility (Standard Deviation): 0.0097


In [43]:
# Step 1: Calculate the Portfolio's Average Daily Return
average_daily_return = maxstock_historical_daily_performance.mean()

# Step 2: Calculate the Portfolio's Volatility (Standard Deviation)
portfolio_volatility = maxstock_historical_daily_performance.std()

# Step 3: Calculate the Total Return of the Portfolio
total_return = maxstock_historical_cumulative[-1] / 1000 - 1

# Print the results
print("Portfolio Total Return: {:.4f}".format(total_return))
print("Portfolio Average Daily Return: {:.4f}".format(average_daily_return))
print("Portfolio Volatility (Standard Deviation): {:.4f}".format(portfolio_volatility))

Portfolio Total Return: 0.4728
Portfolio Average Daily Return: 0.0011
Portfolio Volatility (Standard Deviation): 0.0070


In [44]:
# Step 1: Calculate the Portfolio's Average Daily Return
average_daily_return = sp5_daily_performance.mean()

# Step 2: Calculate the Portfolio's Volatility (Standard Deviation)
portfolio_volatility = sp5_daily_performance.std()

# Step 3: Calculate the Total Return of the Portfolio
total_return = sp5_cumulative[-1] / 1000 - 1

# Print the results
print("Portfolio Total Return: {:.4f}".format(total_return))
print("Portfolio Average Daily Return: {:.4f}".format(average_daily_return))
print("Portfolio Volatility (Standard Deviation): {:.4f}".format(portfolio_volatility))

Portfolio Total Return: 0.2289
Portfolio Average Daily Return: 0.0006
Portfolio Volatility (Standard Deviation): 0.0062
